# Create a single, self-contained Jupyter notebook that implements the
# "single polished notebook" approach for the thesis causal analysis.
# The notebook will:
# - Try to load data from data/cleaned.csv or data/built5.dta
# - If not present, synthesize a small realistic panel dataset so the notebook runs end-to-end
# - Compute period means, surge, define treatment, run DiD OLS with county and year FE and cluster SE
# - Produce an event-study style mean plot
# - Run IV diagnostics: residualize on year FE and scatter residuals
# - Run IV (2SLS) using linearmodels' IV2SLS
# - Create quartile groups and run heterogeneity IV with interacted treatment/instrument
# - All code cells include explanatory comments and printouts
#
# The notebook will be saved to /mnt/data/thesis_single_notebook.ipynb

In [ ]:
import nbformat as nbf
from nbformat.v4 import new_notebook, new_markdown_cell, new_code_cell
import os, json, textwrap

In [ ]:
nb = new_notebook()
nb.cells = []

# Title and overview
nb.cells.append(new_markdown_cell(
"# Reproducible Mini-Analysis: Causal Effect of Procurement on Regional Innovation\n\n"
"This notebook provides a polished, self-contained demonstration of the key analysis from the thesis. "
"It is written to run end-to-end: if your cleaned data file is present under `data/cleaned.csv` (or `data/built5.dta`), "
"the notebook will use it. Otherwise the notebook will simulate a small synthetic panel with realistic variable names "
"so the analyses run and the figures/tables render. \n\n**Sections:**\n"
"- Setup & data loading\n- Data construction / cleaning\n- Difference-in-Differences (DiD) exploratory analysis\n- Instrumental Variables (IV) diagnostics and main specs\n- Heterogeneity analysis (grouped IV)\n\n> Note: this notebook is optimized for clarity and correctness (suitable for a portfolio demo). Replace the synthetic data with your cleaned dataset to reproduce real results."
))

# Setup cell: imports and helper functions
nb.cells.append(new_code_cell(
"""# Setup: imports and helpers
import os
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
from linearmodels.iv import IV2SLS
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid', context='notebook')

# helper functions
def ensure_dir(path):
    d = os.path.dirname(path)
    if d and not os.path.exists(d):
        os.makedirs(d, exist_ok=True)

def read_data(path_csv='data/cleaned.csv', path_dta='data/built5.dta'):
    if os.path.exists(path_csv):
        print(f'Loading CSV data from {path_csv}')
        return pd.read_csv(path_csv)
    if os.path.exists(path_dta):
        print(f'Loading Stata data from {path_dta}')
        return pd.read_stata(path_dta)
    return None

def save_fig(fig, path):
    ensure_dir(path)
    fig.savefig(path, dpi=150, bbox_inches='tight')
    print('Saved figure to', path)
"""
))

# Data loading / synth cell
nb.cells.append(new_code_cell(
"""# Data load or synthetic data creation
df = read_data()
if df is None:
    print('No data file found. Creating a synthetic example dataset for demo purposes.')
    # Create synthetic panel: 50 counties, years 1965-2003
    rng = np.random.default_rng(12345)
    counties = [f'C{i:03d}' for i in range(1,51)]
    years = np.arange(1965, 2004)
    rows = []
    for c in counties:
        base_patents = rng.poisson(5)  # baseline patents
        comp_wt = rng.random()
        for y in years:
            # total dollars grows over time with noise
            total_dollars = max(0.0, 100 + 2*(y-1965) + rng.normal(0,50))
            # create ltotal_dollars (log)
            ltotal = np.log(total_dollars + 1)
            # instrument variation
            lspend_iv = max(0.1, 50 + (y-1975)*rng.normal(0.5,0.5))
            log_sp_iv5 = np.log(lspend_iv + 1)
            # patent counts / citations (noisy)
            num_patents = np.random.poisson( base_patents * (1 + 0.01*(y-1975)) )
            # weighted citations (continuous)
            w_cites_sub = base_patents * 0.5 + rng.normal(0,1)
            lw_cites_sub = np.log(max(0.1, w_cites_sub) + 1)
            avg_wages = 20000 + rng.normal(0,2000)
            pop = max(1000, int(50000 + rng.normal(0,10000)))
            emp = max(100, int(pop*0.4))
            inventor_share = rng.random()
            tech_emp_share = rng.random()
            emp_share = rng.random()
            rows.append({
                'county_fips': c,
                'fyear': int(y),
                'total_dollars': float(total_dollars),
                'ltotal_dollars': float(ltotal),
                'lspending_6675_iv': float(lspend_iv),
                'log_spending_iv5': float(log_sp_iv5),
                'num_patents': int(num_patents),
                'w_cites_sub': float(w_cites_sub),
                'lw_cites_sub': float(lw_cites_sub),
                'avg_wages': float(avg_wages),
                'pop': int(pop),
                'emp': int(emp),
                'comp_wtd': float(comp_wt),
                'inventor_share': float(inventor_share),
                'tech_emp_share': float(tech_emp_share),
                'emp_share': float(emp_share),
                # semi_intens indicator for sample selection
                'semi_intens': int(rng.choice([0,1], p=[0.2, 0.8]))
            })
    df = pd.DataFrame(rows)
    print('Synthetic dataset created with rows:', len(df))
else:
    print('Data loaded with rows:', len(df))
"""))

# Compute period means and surge
nb.cells.append(new_code_cell(
"""# Compute period means (1976-1981 and 1981-1989) and surge per county
df = df.copy()
# compute mean_spend1 and mean_spend2 per county
def mean_cond(series, years, fyear):
    mask = (fyear >= years[0]) & (fyear <= years[1])
    return series.where(mask).mean()

# Use groupby apply to compute per-county means and broadcast
df['mean_spend1'] = df.groupby('county_fips').apply(lambda g: g.loc[g['fyear'].between(1976,1981), 'total_dollars'].mean()).reindex(df['county_fips']).values
df['mean_spend2'] = df.groupby('county_fips').apply(lambda g: g.loc[g['fyear'].between(1981,1989), 'total_dollars'].mean()).reindex(df['county_fips']).values

# surge = mean_spend2 - mean_spend1
df['surge'] = df['mean_spend2'] - df['mean_spend1']

# Summary stats
print('Surge: count non-null =', df['surge'].dropna().shape[0])
print(df[['county_fips','mean_spend1','mean_spend2','surge']].drop_duplicates().head())

# Define treated by median surge across counties (as in your Stata)
county_surge = df.drop_duplicates(subset=['county_fips'])[['county_fips','surge']].dropna().set_index('county_fips')['surge']
surge_median = county_surge.median()
surge_mean = county_surge.mean()
surge_sd = county_surge.std()
print(f"surge_median={surge_median:.3f}, surge_mean={surge_mean:.3f}, surge_sd={surge_sd:.3f}")
df['treated'] = df['county_fips'].map((county_surge > surge_median).astype(int)).fillna(0).astype(int)
# Create after indicator (post-1981)
df['after'] = (df['fyear'] > 1981).astype(int)
df['treatment'] = df['treated'] * df['after']
df.head().iloc[:, :12]
"""))

# Run DiD regression
nb.cells.append(new_code_cell(
"""# Difference-in-Differences OLS with county and year fixed effects and cluster robust SE
# Outcome: w_cites_sub (citation-weighted patent count), treatment: treatment
import statsmodels.formula.api as smf

# select sample as in Stata (keep if semi_intens == 1)
df_did = df[df['semi_intens'] == 1].copy()
print('DiD sample rows:', len(df_did))

formula = 'w_cites_sub ~ treatment + C(county_fips) + C(fyear)'
model = smf.ols(formula=formula, data=df_did)
# cluster by county_fips
res = model.fit(cov_type='cluster', cov_kwds={'groups': df_did['county_fips']})
print(res.summary().tables[1])
"""))

# Event study plot (mean by year for treated vs control)
nb.cells.append(new_code_cell(
"""# Event-study style mean trends (treated vs control)
agg = df_did.groupby(['fyear','treated'])['w_cites_sub'].mean().reset_index()
plt.figure(figsize=(8,4))
for t, grp in agg.groupby('treated'):
    plt.plot(grp['fyear'], grp['w_cites_sub'], marker='o', label=f'treated={int(t)}')
plt.axvline(x=1981, color='k', linestyle='--', label='policy year 1981')
plt.xlabel('Year'); plt.ylabel('Mean citation-weighted patents'); plt.title('Event-study mean trends (treated vs control)')
plt.legend()
plt.tight_layout()
plt.show()
"""))

In [ ]:
# IV diagnostics: residualize on year FE and scatter residuals
nb.cells.append(new_code_cell(
"""# IV First-stage diagnostics: residualize ltotal_dollars and instrument on year FE for subset
mask = (df['semi_intens'] == 1) & (df['fyear'] > 1975) & (df['ltotal_dollars'] > 0) & (df['lspending_6675_iv'] > 0)
print('IV diag rows:', mask.sum())
if mask.sum() > 0:
    sub = df.loc[mask].copy()
    # residualize on year FE: regress variable on C(fyear) and take residuals
    res1 = smf.ols('ltotal_dollars ~ C(fyear)', data=sub).fit()
    sub['resid_lt'] = res1.resid
    res2 = smf.ols('lspending_6675_iv ~ C(fyear)', data=sub).fit()
    sub['resid_sp'] = res2.resid
    # scatter residuals and fitted line
    plt.figure(figsize=(6,5))
    plt.scatter(sub['resid_sp'], sub['resid_lt'], s=10, alpha=0.6)
    # fit line
    coef = np.polyfit(sub['resid_sp'].fillna(0), sub['resid_lt'].fillna(0), 1)
    xs = np.linspace(sub['resid_sp'].min(), sub['resid_sp'].max(), 50)
    plt.plot(xs, coef[0]*xs + coef[1], color='red')
    plt.xlabel('Residuals of instrument (lspending_6675_iv)'); plt.ylabel('Residuals of ltotal_dollars')
    plt.title('Partial relationship: residuals (first-stage diagnostic)')
    plt.tight_layout()
    plt.show()
else:
    print('No rows satisfy IV diagnostic mask; check data.')
"""))

# Run IV 2SLS main spec
nb.cells.append(new_code_cell(
"""# IV 2SLS: example main specification
# We'll run a basic IV where lw_cites_sub is outcome and ltotal_dollars is endogenous,
# instrumented by lspending_6675_iv, including year and county FE and clustering by county.

# Select sample analogous to Stata: semi_intens == 1 & fyear > 1975
df_iv = df[(df['semi_intens'] == 1) & (df['fyear'] > 1975)].copy()
print('IV sample rows:', len(df_iv))

# Build formula for IV2SLS (linearmodels): outcome ~ exog + [endog ~ instr]
# We'll include avg_wages, pop, emp as exogenous controls in the full spec.
formula_iv = 'lw_cites_sub ~ 1 + avg_wages + pop + emp + C(fyear) + C(county_fips) + [ltotal_dollars ~ lspending_6675_iv]'

try:
    iv_mod = IV2SLS.from_formula(formula_iv, data=df_iv)
    iv_res = iv_mod.fit(cov_type='clustered', clusters=df_iv['county_fips'])
    print(iv_res.summary)
except Exception as e:
    print('IV estimation failed (likely due to small synthetic sample or collinearity). Error:', e)
    iv_res = None
"""))

In [ ]:

# Heterogeneity: create quartile groups and run grouped IV
nb.cells.append(new_code_cell(
"""# Heterogeneity analysis by quartiles of comp_wtd (creates xg/zg interactions and runs IV)
df_het = df.copy()
if 'comp_wtd' not in df_het.columns:
    print('comp_wtd not found; creating a synthetic comp_wtd.')
    df_het['comp_wtd'] = np.random.rand(len(df_het))

# keep semi_intens sample (as original Stata)
df_het = df_het[df_het['semi_intens'] == 1].copy()

# Create quartile group (1..4)
df_het['group'] = pd.qcut(df_het['comp_wtd'], q=4, labels=False) + 1

# Create interactions xg1..xg4 and zg1..zg4
for i in range(1,5):
    df_het[f'g{i}'] = (df_het['group'] == i).astype(int)
    df_het[f'xg{i}'] = df_het['ltotal_dollars'] * df_het[f'g{i}']
    df_het[f'zg{i}'] = df_het['log_spending_iv5'] * df_het[f'g{i}']

# Build IV formula: lw_cites_sub ~ exog + FE + [xg1 + ... + xg4 ~ zg1 + ... + zg4]
exog = ['avg_wages','pop','emp']
exog_str = ' + '.join(exog)
endog_terms = ' + '.join([f'xg{i}' for i in range(1,5)])
instr_terms = ' + '.join([f'zg{i}' for i in range(1,5)])
formula_het = f"lw_cites_sub ~ 1 + {exog_str} + C(fyear) + C(county_fips) + [{endog_terms} ~ {instr_terms}]"

print('Heterogeneity IV formula:', formula_het)
try:
    het_mod = IV2SLS.from_formula(formula_het, data=df_het)
    het_res = het_mod.fit(cov_type='clustered', clusters=df_het['county_fips'])
    print(het_res.summary)
    # Extract coefficients for xg1..xg4
    coefs = []
    ses = []
    labels = []
    for i in range(1,5):
        name = f'xg{i}'
        if name in het_res.params.index:
            labels.append(f"q{i}")
            coefs.append(float(het_res.params[name]))
            ses.append(float(het_res.std_errors[name]))
    # Plot coefficients with 95% CI
    if len(coefs) > 0:
        plt.figure(figsize=(6,4))
        x = np.arange(len(coefs))
        plt.errorbar(x, coefs, yerr=[ [1.96*s for s in ses], [1.96*s for s in ses] ], fmt='o', capsize=5)
        plt.xticks(x, labels)
        plt.axhline(0, color='k', linestyle='--')
        plt.xlabel('Group'); plt.ylabel('Estimated Treatment Effect'); plt.title('Heterogeneity IV estimates (by quartile)')
        plt.tight_layout()
        plt.show()
except Exception as e:
    print('Heterogeneity IV failed:', e)
"""))

# Interpretation cell
nb.cells.append(new_markdown_cell(
"## Interpretation & Next Steps\n\n"
"- The notebook runs a clean DiD and IV analysis on a panel dataset. Replace the synthetic data with your real `data/cleaned.csv` to reproduce actual thesis results.\n"
"- If IV estimation fails on the synthetic sample due to small/collinear data, it should run on your real dataset. Check variable names and adjust formula strings accordingly.\n"
"- To increase reproducibility later, move the functions into `src/` modules and add `requirements.txt` + a `scripts/` entrypoint.\n\n"
"---\n"
"**If you want, I can now:**\n"
"1. Convert this notebook into a Colab-ready version (add a badge and small instructions), or\n"
"2. Extract the main tables/figures into `results/` files and add a short README.\n\n"
"Which would you like next?"
))

# Save notebook
out_path = "/mnt/data/thesis_single_notebook.ipynb"
with open(out_path, 'w') as f:
    nbf.write(nb, f)

out_path